### 191_AED


* [issue 191](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/191)
* [this notebook](https://github.com/salgo60/Stockholm_Archipelago_Trail/tree/main/notebook/191_AED.ipynb)

In [1]:
import time
import datetime  
start_time = time.time()
start_str = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
print(f"Started: {start_str}")


Started: 2025-09-26 09:39


In [2]:
#!/usr/bin/env python3
"""
Generisk OSM↔Wikidata-koll via valfri SPARQL

Syfte
------
Skicka in *vilken SPARQL-fråga som helst* som returnerar ett ?item (Wikidata-objekt)
och **någon** form av OSM-id (node/way/relation) – t.ex. via standardegenskaperna
P10689 (node), P11693 (way), P402 (relation) eller som färdiga URL:er – så kontrollerar
skriptet att **motsvarande OSM-objekt pekar tillbaka** till Wikidata via `wikidata=Q…`.

Vad gör skriptet?
-----------------
1) Kör din SPARQL, tolkar resultatet och extraherar:
   - QID (från ?item)
   - OSM-ID:n (node/way/relation) från valfria fält/format.
2) Hämtar nämnda OSM-objekt direkt via Overpass (per id, ej bbox).
3) Validerar back-link:
   - ✅ **OK**: OSM har `wikidata`-tagg som *innehåller* samma QID (hanterar även semikolon-separerade listor).
   - ⚠️ **SAKNAS**: OSM saknar `wikidata`-tag helt.
   - ❌ **FEL**: OSM har `wikidata`, men den matchar **inte** QID.
4) Skriver ut **listor med klickbara länkar** för varje kategori och exporterar CSV.

Exempelanrop
------------
- `run_generic(SPARQL_QUERY)`

Notera
------
- Du kan fortfarande använda den tidigare `run()` för den hårdkodade SAT-frågan om du vill.
"""
from __future__ import annotations
import re
import math
import json
from typing import Dict, Iterable, List, Tuple

import pandas as pd
import requests

WIKIDATA_SPARQL = "https://query.wikidata.org/sparql"
OVERPASS_API = "https://overpass-api.de/api/interpreter"
USER_AGENT = "SAT-OSM-WD-Consistency/2.0 (contact: your-email@example.com)"

# -------------------------
# Hjälpfunktioner
# -------------------------

def make_headers() -> Dict[str, str]:
    return {"User-Agent": USER_AGENT}


def to_osm_prefixed_id(typ: str, osm_id: int) -> str:
    return {"node": "n", "way": "w", "relation": "r"}[typ] + str(int(osm_id))


def parse_osm_from_value(val: str) -> Tuple[str, int] | None:
    """Försök tolka ett OSM-id (typ, id) från godtycklig sträng/URL/tal.
    Stödjer t.ex. 12345, "12345", "https://www.openstreetmap.org/node/12345".
    Returnerar (typ, id) eller None.
    """
    if val is None:
        return None
    s = str(val).strip()

    # URL-format
    m = re.search(r"openstreetmap\.org\/(node|way|relation)\/(\d+)", s)
    if m:
        return m.group(1), int(m.group(2))

    # Rena tal: lämna typ okänd
    if s.isdigit():
        # Typ avgörs av fältnamnet – hanteras av anroparen
        return ("unknown", int(s))

    return None


# -------------------------
# Wikidata
# -------------------------

def fetch_wikidata_generic(sparql: str) -> pd.DataFrame:
    """Kör godtycklig SPARQL och extraherar QID och OSM-id:n ur valfria kolumner.
    Krav: ?item måste finnas och vara ett WD-objekt.
    Heuristik för OSM:
      - Leta kolumnnamn som innehåller något av: node|way|rel|osm|OSM
      - För varje sådan kolumn: tolka värdet som URL eller siffra.
      - Om typ är "unknown" – gissa typ utifrån kolumnnamn (node/way/relation).
    Returnerar DF med kolumner: qid, label (om finns), osm_type, osm_id
    (en rad per QID×OSM-referens). Om ingen OSM-referens hittas för en QID
    returneras ändå en rad utan OSM-typ/id för spårbarhet.
    """
    r = requests.get(WIKIDATA_SPARQL, params={"query": sparql, "format": "json"}, headers=make_headers())
    r.raise_for_status()
    rows = r.json()["results"]["bindings"]

    out_rows: List[Dict] = []
    for b in rows:
        item_uri = b.get("item", {}).get("value")
        if not item_uri:
            continue
        qid = item_uri.rsplit("/", 1)[-1]
        label = b.get("itemLabel", {}).get("value")

        # samla alla kandidater till OSM-kolumner
        osm_keys = [k for k in b.keys() if re.search(r"(node|way|rel|osm)", k, re.I) and k != "item"]

        found_any = False
        for k in osm_keys:
            raw = b[k].get("value")
            parsed = parse_osm_from_value(raw)
            if parsed:
                typ, oid = parsed
                if typ == "unknown":
                    # gissa från kolumnnamn
                    lk = k.lower()
                    if "node" in lk:
                        typ = "node"
                    elif "way" in lk:
                        typ = "way"
                    elif "rel" in lk:
                        typ = "relation"
                    else:
                        # kan ej avgöra – hoppa
                        continue
                out_rows.append({
                    "qid": qid, "label": label, "osm_type": typ, "osm_id": int(oid),
                    "osm_url": f"https://www.openstreetmap.org/{typ}/{int(oid)}",
                })
                found_any = True

        if not found_any:
            # lägg en rad utan OSM för spårbarhet
            out_rows.append({"qid": qid, "label": label, "osm_type": None, "osm_id": None, "osm_url": None})

    df = pd.DataFrame(out_rows).drop_duplicates()
    return df


# -------------------------
# Overpass
# -------------------------

def fetch_osm_by_ids(df_refs: pd.DataFrame) -> pd.DataFrame:
    """Hämta OSM-objekt per id för de rader som har osm_type/osm_id.
    Returnerar kolumner: osm_type, osm_id, url, wikidata_tag
    """
    subset = df_refs.dropna(subset=["osm_type", "osm_id"]).copy()
    if subset.empty:
        return pd.DataFrame(columns=["osm_type","osm_id","url","wikidata_tag"])   

    ids_by_type: Dict[str, List[int]] = {"node": [], "way": [], "relation": []}
    for _, r in subset.iterrows():
        t = r["osm_type"]
        if t in ids_by_type:
            ids_by_type[t].append(int(r["osm_id"]))

    def batched(lst: List[int], n: int = 200) -> Iterable[List[int]]:
        for i in range(0, len(lst), n):
            yield lst[i:i+n]

    rows: List[Dict] = []
    for t in ("node","way","relation"):
        if not ids_by_type[t]:
            continue
        for batch in batched(sorted(set(ids_by_type[t]))):
            ids_str = ",".join(str(i) for i in batch)
            q = f"""
            [out:json][timeout:50];
            {t}(id:{ids_str});
            out ids tags center;
            """.strip()
            rr = requests.post(OVERPASS_API, data={"data": q}, headers=make_headers())
            rr.raise_for_status()
            for el in rr.json().get("elements", []):
                tags = el.get("tags", {})
                rows.append({
                    "osm_type": t,
                    "osm_id": el.get("id"),
                    "url": f"https://www.openstreetmap.org/{t}/{el.get('id')}",
                    "wikidata_tag": tags.get("wikidata"),
                    "name": tags.get("name"),
                })

    return pd.DataFrame(rows)


# -------------------------
# Validering & utskrift
# -------------------------

def normalize_wikidata_values(val: str | None) -> List[str]:
    if not val:
        return []
    # Dela på semikolon/komma och trimma, ta bara sådant som ser ut som Q…
    parts = re.split(r"[;|,]", val)
    return [p.strip() for p in parts if re.match(r"^Q\d+$", p.strip(), re.I)]


def verify_backlinks(df_refs: pd.DataFrame, df_osm: pd.DataFrame) -> Dict[str, pd.DataFrame]:
    # join på (osm_type, osm_id)
    key = ["osm_type","osm_id"]
    merged = pd.merge(df_refs.dropna(subset=key), df_osm, on=key, how="left", suffixes=("","_osm"))

    ok_rows = []
    missing_rows = []
    wrong_rows = []

    for _, r in merged.iterrows():
        qid = r["qid"]
        tag = r.get("wikidata_tag")
        if pd.isna(tag) or not tag:
            missing_rows.append(r)
        else:
            vals = normalize_wikidata_values(tag)
            if qid in vals:
                ok_rows.append(r)
            else:
                wrong_rows.append(r)

    return {
        "ok": pd.DataFrame(ok_rows),
        "missing": pd.DataFrame(missing_rows),
        "wrong": pd.DataFrame(wrong_rows),
        "all_refs": df_refs,
        "all_osm": df_osm,
    }


def print_link_lists(res: Dict[str, pd.DataFrame]):
    def print_group(title: str, df: pd.DataFrame):
        print(f"\n{title}")
        print("-" * len(title))
        if df.empty:
            print("(inget)")
            return
        # Grupp: per QID
        for qid, grp in df.groupby("qid"):
            print(f"{qid}:")
            for _, r in grp.iterrows():
                print(f"  {r['url']}")

    print_group("❌ OSM har wikidata men pekar fel", res["wrong"])        
    print_group("✅ OSM back-link OK (wikidata matchar QID)", res["ok"])    
    print_group("⚠️ OSM saknar wikidata-tag (lägg till QID)", res["missing"]) 


# -------------------------
# Publika körfunktioner
# -------------------------

def run_generic(sparql_query: str, export_prefix: str = "wd_osm_backlinks"):
    print("Kör SPARQL…")
    refs = fetch_wikidata_generic(sparql_query)
    print(f"Rader från SPARQL (QID×OSM-ref): {len(refs)}")

    print("Hämtar OSM per id…")
    osm = fetch_osm_by_ids(refs)
    print(f"OSM-objekt hämtade: {len(osm)}")

    res = verify_backlinks(refs, osm)

    # Exportera CSV
    for name, df in res.items():
        path = f"{export_prefix}_{name}.csv"
        df.to_csv(path, index=False)
        print(f"↳ sparade {path} ({len(df)} rader)")

    # Listrapport med klickbara länkar
    print_link_lists(res)



def run_sat_example():
    run_generic(SAT_SPARQL, export_prefix="sat_backlinks")


In [3]:
# https://w.wiki/FT6j
run_generic("""
SELECT  ?item  ?itemLabel  ?coord ?img ?OSMnode ?OSMway ?OSMrel WHERE {
  ?item wdt:P31 wd:Q1450682;        # AED kopplade till SAT i WD
       wdt:P6104 wd:Q134294510
    OPTIONAL  { ?item wdt:P18 ?img. } # Bild
    OPTIONAL { ?item wdt:P625 ?coord }
    OPTIONAL { ?item wdt:P10689 ?OSMway }
    OPTIONAL { ?item wdt:P11693 ?OSMnode }
    OPTIONAL { ?item wdt:P402   ?OSMrel }

    SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }

}
ORDER BY ?itemlabel
""", export_prefix="sat_backlinks")


Kör SPARQL…
Rader från SPARQL (QID×OSM-ref): 25
Hämtar OSM per id…
OSM-objekt hämtade: 25
↳ sparade sat_backlinks_ok.csv (22 rader)
↳ sparade sat_backlinks_missing.csv (3 rader)
↳ sparade sat_backlinks_wrong.csv (0 rader)
↳ sparade sat_backlinks_all_refs.csv (25 rader)
↳ sparade sat_backlinks_all_osm.csv (25 rader)

❌ OSM har wikidata men pekar fel
--------------------------------
(inget)

✅ OSM back-link OK (wikidata matchar QID)
-----------------------------------------
Q134568948:
  https://www.openstreetmap.org/node/12804717601
Q135010859:
  https://www.openstreetmap.org/node/12804831701
Q135012478:
  https://www.openstreetmap.org/node/12942128203
Q135013540:
  https://www.openstreetmap.org/node/12805751201
Q135013563:
  https://www.openstreetmap.org/node/12805053401
Q135013584:
  https://www.openstreetmap.org/node/12809454601
Q135013590:
  https://www.openstreetmap.org/node/12809240302
Q135013597:
  https://www.openstreetmap.org/node/12805742901
Q135013611:
  https://www.openstree

In [4]:
# =========================
# KONFIG
# =========================
SAT_GEOJSON_PATH   = "SAT_full.geojson"
OUTPUT_PREFIX      = "191_WD_AED"
FETCH_FROM_WD      = True

# https://w.wiki/FT6x
SPARQL = r"""
SELECT ?item ?itemLabel ?coord ?img ?OSMnode ?OSMway ?OSMrel 
     ?type ?typeLabel ?kartIcon ?hex
WHERE {
  ?item wdt:P31 wd:Q1450682;        # AED kopplade till SAT i WD
       wdt:P6104 wd:Q134294510.
  ?item wdt:P31 ?type.

  OPTIONAL { ?item wdt:P625 ?coord. }
  OPTIONAL { ?item wdt:P18 ?img. }
  OPTIONAL { ?item wdt:P11693 ?OSMnode }
  OPTIONAL { ?item wdt:P10689 ?OSMway }
  OPTIONAL { ?item wdt:P402   ?OSMrel }

  # fetch Kartographer info from type
  OPTIONAL { ?type wdt:P1343 ?kartIcon. }
  OPTIONAL { ?type wdt:P465 ?hex. }

  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
ORDER BY ?itemLabel
"""


In [9]:
def fetch_wd_to_csv(sparql: str, out_csv: str):
    url = "https://query.wikidata.org/sparql"
    headers = {
        "Accept": "application/sparql-results+json",
        "User-Agent": "SAT-map-builder/1.2 (contact: salgo60@msn.com)"
    }
    r = requests.get(url, params={"query": sparql}, headers=headers, timeout=80)
    r.raise_for_status()
    data = r.json()

    rows = []
    for b in data["results"]["bindings"]:
        gv = lambda k: b[k]["value"] if k in b else None
        rows.append({
            "item": gv("item"),
            "itemLabel": gv("itemLabel"),
            "coord": gv("coord"),
            "img": gv("img"),
            "OSMnode": gv("OSMnode"),
            "OSMway": gv("OSMway"),
            "OSMrel": gv("OSMrel"),
        })

    df = pd.DataFrame(rows)

    # parse coord "Point(lon lat)"
    lats, lons = [], []
    for v in df["coord"].astype(str):
        lat = lon = None
        if v.startswith("Point(") and v.endswith(")"):
            parts = v[6:-1].split()
            if len(parts) == 2:
                try:
                    lon = float(parts[0]); lat = float(parts[1])
                except:
                    pass
        lats.append(lat); lons.append(lon)
    df["lat"] = lats
    df["lon"] = lons
    df.to_csv(out_csv, index=False)
    return df


In [11]:
if FETCH_FROM_WD:
    print("Hämtar AED från Wikidata…")
    df_ferry = fetch_wd_to_csv(SPARQL, "AED_wd.csv")
else:
    if not os.path.exists("AED_wd.csv"):
        raise FileNotFoundError("Hittar inte AED_wd.csv – kör FETCH_FROM_WD=True eller lägg filen bredvid scriptet.")
    df_ferry = pd.read_csv("AED_wd.csv")

print(f"AED: {len(df_ferry)}")


Hämtar AED från Wikidata…
AED: 25
